# Исследование надежности заемщиков.

## **Цель исследования:**
Выявление взаимосвязи между фактом возврата кредита в срок и определенными факторами:
1. семейное положение 
2. количество детей



***Описание проекта***  
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.  
Входные данные от банка — статистика о платёжеспособности клиентов.  
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

***Описание данных***  
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

### Шаг 1. Обзор данных

In [5]:
import pandas as pd
try:
    df = pd.read_csv("D:/practicum/datasets_for_projects/data.csv")
except:
    df = pd.read_csv("/datasets/data.csv")
df.info()
df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Беглый взгляд на датасет позволяет сделать первичные выводи и выдвинуть некоторые предположения.
1. с типами данных по столбцам всё в порядке, возможно будет удобно привести столбцы с вещественными числами к целочисленному типу.
2. наименования столбцов корректные, единственное, что хочется заменить - это dob-years на age для удобства.
3. Пропуски присутствуют в двух столбцах **days_employed** и **total_income** и в равном количестве. Стоит проверить, совпадают ли строки с пропусками по этим столбцам
4. Так же бросаются в глаза отрицательные значения трудового стажа 
5. и разница в регистре значений столбца education

In [6]:
#переименовываем столбец
df = df.rename(columns={"dob_years" : "age"})
df.columns

Index(['children', 'days_employed', 'age', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [7]:
#проверим совподают ли строки спропущенными значениями в двух столбцах
display(df[df["days_employed"].isna()])
display(df[df["days_employed"].isna()]["total_income"].isna().sum())

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


2174

Мы видим, что строки с пропущенными значениями в двух столбцах совпадают посчитаем процент пропусков

In [8]:
df["days_employed"].isna().mean()

0.10099883855981417

10% - значительная доля, но для основной цели исследования столбцы с пропусками не важны, значит удалять строки нецелесообразно. можно заполнить пропуски средним или медианным значением, но сначала нужно разобраться с некорректными значениями. рассмотрим столбец с трудовым стажем

### Шаг 2.1 Проверка данных на аномалии и исправления.

In [9]:
#отсартируем по стажу и посмотрим на минимальные и максимальные значения
display(df.sort_values(by="days_employed", ascending=False).head(20))
df.sort_values(by="days_employed").head(20)


,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
4697,0,401635.032697,56,среднее,1,женат / замужем,0,F,пенсионер,0,48242.322502,покупка недвижимости
13420,0,401619.633298,63,Среднее,1,гражданский брак,1,F,пенсионер,0,51449.788325,сыграть свадьбу
17823,0,401614.475622,59,среднее,1,женат / замужем,0,F,пенсионер,0,152769.694536,покупка жилья для сдачи
10991,0,401591.828457,56,среднее,1,в разводе,3,F,пенсионер,0,39513.517543,получение дополнительного образования
8369,0,401590.452231,58,среднее,1,женат / замужем,0,F,пенсионер,0,175306.312902,образование


,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,-17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,-16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,-16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,-16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости
3974,0,-15835.725775,64,среднее,1,гражданский брак,1,F,компаньон,0,96858.531436,сыграть свадьбу
1539,0,-15785.678893,59,высшее,0,Не женат / не замужем,4,F,сотрудник,0,119563.851852,операции с коммерческой недвижимостью
4321,0,-15773.061335,61,среднее,1,гражданский брак,1,F,сотрудник,0,205868.585780,свадьба
7731,0,-15618.063786,64,среднее,1,женат / замужем,0,F,компаньон,0,296525.358574,высшее образование
15675,0,-15410.040779,65,высшее,0,женат / замужем,0,F,сотрудник,0,188800.068859,покупка жилой недвижимости


В глаза бросается закономерность: положительные значения стажа только у пенсионеров, у остальных отрицательные. Похоже, что отрицательные значения возникли из-за отсутствия даты окончания работы при заполнении анкет. проверим эту гипотезу

In [10]:
#посмотрим на уникальные значения рабочего статуса для положительных значений стажа
display(df.loc[df["days_employed"] > 0, "income_type"].unique())  

array(['пенсионер', 'безработный'], dtype=object)

In [11]:
#посмотрим на безработных
unemployed = df[df["income_type"] == "безработный"]
display(unemployed["income_type"].count())
unemployed

2

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,395302.838654,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


Гипотеза подтвердилась. 
Теперь переведем дни в годы, чтобы проверить правдоподобность данных, и добавим столбец.
отдельно отсартируем положительные и отрицательные значения

In [12]:
#добавим столбец со стажем в годах
df["years_employed"] = df["days_employed"]/365
# для удобства устраним лишние столбцы
df_temp = df[["age", "days_employed", "years_employed"]]
#посмотрим на получившиеся положительные и отрицательные значения
display(df_temp[df_temp["days_employed"] >= 0].sort_values(by="days_employed", ascending=False).head(10))
display(df_temp[df_temp["days_employed"] >= 0].sort_values(by="days_employed").head(10))
display(df_temp[df_temp["days_employed"] < 0].sort_values(by="days_employed", ascending=False).head(10))
display(df_temp[df_temp["days_employed"] < 0].sort_values(by="days_employed").head(10))
#заодно проверим нет ли нулевых значений
display(df[df["days_employed"] == 0]["days_employed"].count())

,age,days_employed,years_employed
6954,56,401755.400475,1100.699727
10006,69,401715.811749,1100.591265
7664,61,401675.093434,1100.479708
2156,60,401674.466633,1100.477991
7794,61,401663.850046,1100.448904
4697,56,401635.032697,1100.369953
13420,63,401619.633298,1100.327762
17823,59,401614.475622,1100.313632
10991,56,401591.828457,1100.251585
8369,58,401590.452231,1100.247814


,age,days_employed,years_employed
20444,72,328728.720605,900.626632
9328,41,328734.923996,900.643627
17782,56,328771.341387,900.743401
14783,62,328795.726728,900.810210
7229,32,328827.345667,900.896837
5332,55,328834.274455,900.915820
5756,51,328842.364916,900.937986
13325,55,328862.571412,900.993346
3642,54,328874.270668,901.025399
3386,58,328895.862752,901.084555


,age,days_employed,years_employed
17437,31,-24.141633,-0.066141
8336,32,-24.240695,-0.066413
6157,47,-30.195337,-0.082727
9683,43,-33.520665,-0.091837
2127,31,-34.701045,-0.095071
5287,36,-37.726602,-0.103361
17270,34,-39.954170,-0.109463
13846,33,-46.952793,-0.128638
7964,49,-47.109840,-0.129068
3235,43,-50.128298,-0.137338


,age,days_employed,years_employed
16335,61,-18388.949901,-50.380685
4299,61,-17615.563266,-48.261817
7329,60,-16593.472817,-45.461569
17838,59,-16264.699501,-44.560821
16825,64,-16119.687737,-44.163528
3974,64,-15835.725775,-43.385550
1539,59,-15785.678893,-43.248435
4321,61,-15773.061335,-43.213867
7731,64,-15618.063786,-42.789216
15675,65,-15410.040779,-42.219290


0

1. Нулевых значений нет
2. Максимальные значения по модулю для работающих в целом выглядят достаточно правдоподобно в соотношении с возрастом, а судя по соотношению возраста и минимальных значений, можно предположить, что учитывалось только последнее место работы. Отрицательные, но корректные по модулю значения могли получиться в результате вычитания текущей даты из даты начала работы. 
3. Разброс значений среди пенсионеров от 900 до 1100 лет, значит, вероятнее всего, мы имеем дело с общей ошибкой в подсчетах. возможно их стаж перевели в часы, а не в дни

Проверим эту гипотезу

In [13]:
# переведем значение в столбце предположительно в годы
(df[df["years_employed"] > 0]["years_employed"]/24).head(20)

4      38.843159
18     45.694194
24     38.647483
25     41.500969
30     38.308410
35     44.979574
50     40.380300
56     42.254005
71     38.597435
78     41.064263
86     43.828031
87     38.545609
88     42.594725
98     41.655960
99     39.559546
101    44.698512
104    41.817768
119    43.628896
128    43.268941
129    41.211224
Name: years_employed, dtype: float64

Такие данные стажа пенсионеров выглядят значительно правдоподобнее. 

In [14]:
#заменим их
df.loc[df["days_employed"] >= 0,"years_employed"] = df["years_employed"]/24

Проверим средние и медианные значения для возраста и стажа работающих и пенсионеров.

In [15]:
print("работающие".upper())
print("средний возраст:")
display(df[df["days_employed"] < 0]["age"].mean())
print("средний стаж:")
display(-1*(df[df["days_employed"] < 0]["years_employed"].mean()))
print("медианный стаж:")
display(-1*(df[df["days_employed"] < 0]["years_employed"].median()))
print("пенсионеры".upper())
print("средний возраст:")
display(df[df["days_employed"] >= 0]["age"].mean())
print("средний стаж:")
display(df[df["days_employed"] >= 0]["years_employed"].mean())
print("медианный стаж:")
display(df[df["days_employed"] >= 0]["years_employed"].median())

РАБОТАЮЩИЕ
средний возраст:


39.81824468753929

средний стаж:


6.446618991777744

медианный стаж:


...

59.12481857764877

средний стаж:


41.667158666240674

медианный стаж:


41.69101669700128

Судя по всему, гипотизы верны. <a id='replace'></a>

1. Заменим отрицательные значения положительными на следующем этапе исследования
2. Пропущенные значения стажа среди пенсионеров заменим средним значением, так как разница между средним и медианным значением незначительная, значит нет сильно выделяющихся значений, а в таком случае среднееарифметическое - более точная величина.
3. Так же заменим для пенсионеров значение стажа в днях на корректное.(переведем часы в дни)
4. Пропущенные значения стажа работающих, так как в них нет никакого статистического смысла и логики, я заменю нулем, чтобы они выделялись на фоне остальных как не заполненные ячейки на случай, если придется проводить точечные исследования связанные с этим столбцом в дальнейшем, в замене их медианой не вижу никакого созидательного смысла.

In [16]:
# заменяем значения стажа в днях для пенсионеров и безработных
df.loc[df["days_employed"] > 0, "days_employed"] = df["days_employed"] / 24
# проверим результат
df.loc[df["days_employed"] > 0, "days_employed"].head(10)

4     14177.753002
18    16678.380705
24    14106.331371
25    15147.853723
30    13982.569521
35    16417.544674
50    14738.809681
56    15422.711968
71    14088.063746
78    14988.456045
Name: days_employed, dtype: float64

Прежде чем переходить к заполнению пропусков проверим на неявные дубликаты и аномалии значения категориальных столбцов

In [17]:
print("education".upper())
display(df["education"].unique())
print("education_id".upper())
display(df["education_id"].unique())
print("family_status".upper())
display(df["family_status"].unique())
print("family_status_id".upper())
display(df["family_status_id"].unique())
print("gender".upper())
display(df["gender"].unique())
print("income_type".upper())
display(df["income_type"].unique())
print("debt".upper())
display(df["debt"].unique())
print("purpose".upper())
display(df["purpose"].unique())

EDUCATION


array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

EDUCATION_ID


array([0, 1, 2, 3, 4], dtype=int64)

FAMILY_STATUS


...

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

DEBT


array([0, 1], dtype=int64)

PURPOSE


array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

итак: <a id='anomaly'></a>
1. По колонке с образоанием проблемы только с регистром. приведем значения к нижнему регистру 
2. Количество индексов совпадает с количеством значений в соответствующих столбцах. Думаю проверять соответствие индексов значениям - излишне 
3. Столбец с семейным положением тоже приведем к нижнему регистру для порядка 
4. В столбце с полом присутствует аномальное значение. проверим количество строк с этим значением 
5. Цель кредита описана в свободной форме и требует котегоризации

In [18]:
# Приводим к нижнему регистру значения в столбцах "education" и "family_status"
df["education"] = df["education"].str.lower()
display(df["education"].unique())

df["family_status"] = df["family_status"].str.lower()
display(df["family_status"].unique())
# проверим количество некорректных значения в колонке с полом
display(df["gender"].value_counts())
df[df["gender"] == "XNA"]

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости,-6.461919


Всего одно аномальное значение пола. Так как достоверной информации по данной ячейке нет и пол не играет роли в основном исследовании, удалять строку бессмыслено, поэтому оставлю позицию без изменений

Теперь рассмотрим последний столбец играющий ключевую роль в исследовании

In [19]:
display(df["children"].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

в списке значений столбца присутствует 2 явно аномальных значения: 20 и -1. 
20, вероятнее всего, - опечатка на цифровой клавиатуре, где 2 и 0 расположены рядом. Она может означать и 2 и 0, как и -1 может означать и 1 и "прочерк". 
Полагаю, учитывая, очевидно, крайне незначительный процент аномальных значений, для большей достоверности исследования строки с этими значениями стоит удалить

In [20]:
df = df[(df["children"]!= -1) & (df["children"]!= 20)]
# проверим результат удаления
df[df["children"] == -1]["children"].count()

0

### Шаг 2.2 Заполнение пропусков

[Вернемся к столбцам со стажем](#replace)

In [21]:
#создаем переменные медиан для пенсионеров
unemployed_days_median = df[df["days_employed"] > 0]["days_employed"].median()
unemployed_years_median = df[df["years_employed"] > 0]["years_employed"].median()
#заменяем все пропущенные значения на ноль
df["days_employed"] = df["days_employed"].fillna(0)
df["years_employed"] = df["years_employed"].fillna(0)
#заменяем нулевые значения пенсионеров на медиану
df.loc[(df["income_type"] == "пенсионер") & (df["days_employed"] == 0), "days_employed"] = unemployed_days_median
df.loc[(df["income_type"] == "пенсионер") & (df["years_employed"] == 0), "years_employed"] = unemployed_years_median
#замена отрицательных значений
df.loc[df["days_employed"] < 0, "years_employed"] = df["years_employed"] *(-1)
df.loc[df["days_employed"] < 0, "days_employed"] = df["days_employed"] *(-1)
df.head(20)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,38.843159
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,2.537495
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,7.888225
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,0.418574
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,18.985932
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,5.996593


Теперь рассмотрим второй столбец с пропусками

In [22]:
display(df["total_income"].mean())
df["total_income"].median()

167448.78975722924

145020.80127962783

Довольно серьезное расхождение между средним и медианным значением: вероятно, присутствуют слишком выделяющиеся значения.
Средний доход кажется высоковатым, но не уверен, что я компетентен судить об этом, однако есть сомнения в достоверности данных.

In [23]:
# посмотрим максимальное и минимальное значения
df["total_income"].max(), df["total_income"].min()

(2265604.028722744, 20667.26379327158)

Без комментариев.
Заполним пропуски медианой.

In [24]:
# income_median = df["total_income"].median()
# df["total_income"] = df["total_income"].fillna(value=income_median)
# df["total_income"].isna().sum()

<div class="alert alert-block alert-warning">
<b>Комментарий ревьюера⚠️:</b> 
Есть еще один, более интересный способ заполнения пропусков. Например, доход клиента (total_income) можно заполнить медианными значениями в разрезе income_type - для каждого income_type посчитать медианный total_income и заполнить пропуски ими. Так мы не усредним данные и меньше повлияем на распределение)

<div class="alert alert-block alert-info">
согласен. Ща сделаем. Предыдущий вариант я закомментил.

In [25]:
df["income_type"].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [26]:
df["total_income"].isna().sum()

2162

In [27]:
# создаем список медиан
median_list = df.groupby("income_type")["total_income"].median()
median_list

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150420.150276
компаньон          172517.418907
пенсионер          118480.837408
предприниматель    499163.144947
сотрудник          142587.588976
студент             98201.625314
Name: total_income, dtype: float64

In [28]:
# создаем список соответствующих медианам категорий
median_index = median_list.index

# цикл заполняет пропущенные значения медианой по категориям
for i in range(len(median_list)):
    df.loc[(df["income_type"] == median_index[i]) & (df["total_income"].isna()), "total_income"] = median_list[i]
        
df["total_income"].isna().sum()

0

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   age               21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  float64
 11  purpose           21402 non-null  object 
 12  years_employed    21402 non-null  float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.8+ MB


<div class="alert alert-block alert-info">
Честно говоря, я знатно поломал голову над этим. Ничего лучше не придумал

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера_v2✅:</b> Твой способ вполне хорош, я бы сделал так же😉 </div>

### Шаг 2.3. Изменение типов данных.

приведем значения столбцов с вещественными числами к целочисленному типу данных

In [30]:
df["days_employed"] = df["days_employed"].astype("int")
df["years_employed"] = df["years_employed"].astype("int")
df["total_income"] = df["total_income"].astype("int")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21402 non-null  int64 
 1   days_employed     21402 non-null  int32 
 2   age               21402 non-null  int64 
 3   education         21402 non-null  object
 4   education_id      21402 non-null  int64 
 5   family_status     21402 non-null  object
 6   family_status_id  21402 non-null  int64 
 7   gender            21402 non-null  object
 8   income_type       21402 non-null  object
 9   debt              21402 non-null  int64 
 10  total_income      21402 non-null  int32 
 11  purpose           21402 non-null  object
 12  years_employed    21402 non-null  int32 
dtypes: int32(3), int64(5), object(5)
memory usage: 2.5+ MB


<div class="alert alert-block alert-success">
<b>Комментарий ревьюера✅:</b> На этом этапе нечего добавить, все круто👍</div>

### Шаг 2.4. Удаление дубликатов.

In [31]:
display(df.duplicated().sum())
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

71

0

Дубликаты в данных могут появляться из-за ошибок при копировании или из-за человеческого фактора

Неявные дубликаты были устранены вместе с [аномалиями](#anomaly)

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера✅:</b> Молодец, дубликаты удалены верно👍

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [32]:
# создаем таблицу с уникальными значениями "education" и "education_id"
education_log = df[["education_id", "education"]]
education_log = education_log.drop_duplicates().reset_index(drop=True)
display(education_log)
# создаем таблицу с уникальными значениями "family_status" и "family_status_id"
family_status_log = df[["family_status_id", "family_status"]]
family_status_log = family_status_log.drop_duplicates().reset_index(drop=True)
display(family_status_log)
# удаляем из исходного датафрейма столбцы "education" и "family_status"
df = df.drop(["family_status", "education"], axis=1)
# df = df[["children", "days_employed", "age", "education_id", "family_status_id", "gender", "income_type", "debt", "total_income", "purpose"]]
df.head()

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


,children,days_employed,age,education_id,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,23
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,15
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,38


<div class="alert alert-block alert-success">
<b>Комментарий ревьюера✅:</b> Сделано верно, молодец🔥</div>

### Шаг 2.6. Категоризация дохода.

In [33]:
def income_category(total_income):
    """Функция возвращает буквенные значения
    категорий по диапазону заработка"""
    if total_income <= 30000:
        return "E"
    elif total_income <= 50000:
        return "D"
    elif total_income <= 200000:
        return "C"
    elif total_income <= 1000000:
        return "B"
    return "A"
df["total_income_category"] = df["total_income"].apply(income_category)
df.head(10)

,children,days_employed,age,education_id,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,23,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,11,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,15,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,11,B
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,38,C
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,2,B
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,7,B
7,0,152,50,1,0,M,сотрудник,0,135823,образование,0,C
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,18,C
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,5,C


Мы получили новую колонку с категориями дохода

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера✅:</b> Функция написана верно, круто что результат ее выполнения проверен👍

### Шаг 2.7. Категоризация целей кредита.

Посмотрим на уникальные значения целей кредита

In [34]:
purp = df["purpose"].sort_values().unique()
display(purp)

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

Список достаточно большой, поэтому, чтобы не ошибиться и ничего не пропустить, напишем функцию наглядно показывающую распределение.

In [35]:
def simple_division(purpose):
    """функция распределяет уникальные элементы исходного Series 
    по спискам по наличию в них ключевых буквенных сочетаний.
    в конечном результате последний список должен быть пуст"""
    avto = []
    realty = []
    merrage = []
    education = []
    unidentified = [] #список для значений не попавших ни в одну категорию
    for item in purpose:
        if "авто" in item:
            avto.append(item)
        elif "недвиж" in item or "жиль" in item:
            realty.append(item)
        elif "свадьб" in item:
            merrage.append(item)
        elif "образ" in item:
            education.append(item)
        else:
            unidentified.append(item)
    return avto, realty, merrage, education, unidentified
avto, realty, merrage, education, unidentified = simple_division(purp)
simple_division(purp)

(['автомобили',
  'автомобиль',
  'на покупку автомобиля',
  'на покупку подержанного автомобиля',
  'на покупку своего автомобиля',
  'приобретение автомобиля',
  'свой автомобиль',
  'сделка с автомобилем',
  'сделка с подержанным автомобилем'],
 ['жилье',
  'недвижимость',
  'операции с жильем',
  'операции с коммерческой недвижимостью',
  'операции с недвижимостью',
  'операции со своей недвижимостью',
  'покупка жилой недвижимости',
  'покупка жилья',
  'покупка жилья для сдачи',
  'покупка жилья для семьи',
  'покупка коммерческой недвижимости',
  'покупка недвижимости',
  'покупка своего жилья',
  'ремонт жилью',
  'строительство жилой недвижимости',
  'строительство недвижимости',
  'строительство собственной недвижимости'],
 ['на проведение свадьбы', 'свадьба', 'сыграть свадьбу'],
 ['высшее образование',
  'дополнительное образование',
  'заняться высшим образованием',
  'заняться образованием',
  'образование',
  'получение высшего образования',
  'получение дополнительного о

Теперь на основе результатов создадим столбец "purpose_category"

In [36]:
def purpose_cat(purpose):
    if purpose in avto:
        return "операции с автомобилем"
    elif purpose in realty: 
        return "операции с недвижимостью"
    elif purpose in merrage:
        return "проведение свадьбы"
    elif purpose in education:
        return "получение образования"
df["purpose_category"] = df["purpose"].apply(purpose_cat)
# проверим результат
df.head()

,children,days_employed,age,education_id,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,23,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,11,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,15,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,11,B,получение образования
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,38,C,проведение свадьбы


<div class="alert alert-block alert-success">
<b>Комментарий ревьюера✅:</b> Отличная функция, результат ее выполнения проверен, молодец👍

### Шаг 2.8. Исследование зависимости возврата кредита в срок от различных факторов.

In [37]:
# сводная таблица по количеству детей
children_debt = df.pivot_table(index="children", values="debt", aggfunc=("count", "mean"))
children_debt

,count,mean
children,,
0,14091.0,0.075438
1,4808.0,0.092346
2,2052.0,0.094542
3,330.0,0.081818
4,41.0,0.097561
5,9.0,0.000000


самый низкий процент просрочек по кредитам у клиентов без детей и это основное количество клиентов. Количество клиентов с тремя, четырьмя и пятью детьми недостаточно для каких-либо выводов у клиентов с одним и двумя детьми процент просрочек практически идентичен и немного выше, чем у клиентов без детей

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера✅:</b> Ответ верный👍</div>

In [38]:
# сводная таблица по семейному статусу
family_status_debt = df.pivot_table(index="family_status_id", values="debt", aggfunc=("count", "mean"))
display(family_status_debt, family_status_log)

,count,mean
family_status_id,,
0,12261.0,0.075606
1,4134.0,0.093130
2,951.0,0.066246
3,1189.0,0.070648
4,2796.0,0.097639


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Наименьший процент просрочек у вдовцов и вдов и это самая малочисленная группа. хуже всех платят не женатые и не замужние и в гражданском браке. разведенные и женатые приблизительно наравне и платят немного хуже вдовствующих. разброс значений более трех процентов

<div class="alert alert-block alert-danger">
<b>Комментарий ревьюера❌:</b> С ответом не соглашусь😉
    
Представь, что мы выдаем кредиты на миллионы долларов в месяц, при таких масштабах 1.5 - 2% могут сэкономить нам кучу денег😉

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера_v2✅:</b> Теперь согласен👍 </div>

In [39]:
#сводная таблица по уровню заработка
total_income_debt = df.pivot_table(index="total_income_category", values="debt", aggfunc=("count", "mean"))
total_income_debt

,count,mean
total_income_category,,
A,25.0,0.080000
B,5014.0,0.070602
C,15921.0,0.084982
D,349.0,0.060172
E,22.0,0.090909


Четких закономерности в количестве просрочек от категории дохода так же не видно.

In [40]:
df.groupby("purpose_category")["debt"].mean()

purpose_category
операции с автомобилем      0.093480
операции с недвижимостью    0.072551
получение образования       0.092528
проведение свадьбы          0.079118
Name: debt, dtype: float64

In [41]:
# сводная таблица по двум критериям: кол-во детей и семейное положение
final_pivot = df.pivot_table(index="children", columns="family_status_id", values="debt", aggfunc=("count", "mean"))
display(final_pivot, family_status_log)

count                                    mean            \
family_status_id       0       1      2      3       4         0         1   
children                                                                     
0                 7468.0  2730.0  847.0  784.0  2262.0  0.069095  0.083883   
1                 2975.0   995.0   77.0  312.0   449.0  0.082689  0.118593   
2                 1533.0   343.0   20.0   81.0    75.0  0.094586  0.087464   
3                  249.0    56.0    6.0   11.0     8.0  0.068273  0.142857   
4                   29.0     8.0    1.0    1.0     2.0  0.103448  0.000000   
5                    7.0     2.0    NaN    NaN     NaN  0.000000  0.000000   

                                                
family_status_id         2         3         4  
children                                        
0                 0.062574  0.070153  0.092838  
1                 0.090909  0.067308  0.115813  
2                 0.150000  0.086420  0.120000  
3                 0.000000  0.090909  0.125000  
4                 0.000000  0.000000  0.500000  
5                      NaN       NaN       NaN

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


На обобщенной схеме картина несколько иная, но большинство групп слишком малы для объективного анализа 

### Ответы на вопросы.

1. Между колличеством детей и выплатами кредита в срок закономерность не выявлена в связи с малочисленностью большинства групп, но исследование показывает, что бездетные клиенты платят лучше остальных. то есть присутствует зависимость между наличием/отсутствием детей и своевременными выплатами
2. зависимость между семейным положением и выплатами по кредиту присутствует.Наименьший процент просрочек у вдовцов и вдов, хотя это может быть связано с возрастной категорией основной массы группы, хуже всех платят не женатые и не замужние и в гражданском браке. 
3. В группах по уровню дохода присутствуют различия в доле просрочек, но линейной зависимости не выявлено и я считаю, что делать какие-то выводы не этой основе не стоит.



<div class="alert alert-block alert-danger">
<b>Комментарий ревьюера❌:</b> Пересмотри пожалуйста свой вывод с учетом замечания выше)

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера_v2✅:</b> 👍 </div>

## Общий вывод:



В результате исследования были выявлены некоторые закономерности:
1. самый низкий процент просрочек по кредитам у клиентов без детей и это основное количество клиентов. Количество клиентов с тремя, четырьмя и пятью детьми недостаточно для каких-либо выводов у клиентов с одним и двумя детьми процент просрочек практически идентичен и немного выше, чем у клиентов без детей
2. Наименьший процент просрочек у вдовцов и вдов, однако это самая малочисленная группа. хуже всех платят не женатые и не замужние и в гражданском браке. разведенные и женатые приблизительно наравне и платят немного хуже вдовствующих. разброс значений более трех процентов
3. линейной зависимости между уровнем заработка и просроченными выплатами не прослеживается. Четких закономерности в количестве просрочек от категории дохода так же не видно.

Средние значения задержек по выплатам в группах отличаются незначительно. Многие группы слишком малы для анализа.
Единственный вывод, который можно сделать, что среди групп значительного размера, наименьший процент просрочек у женатых и вдовствующих клиентов без детей.
Думаю, что для поиска закономерностей между просроченными платежами и уровнем достатка нехватает нескольких ключевых метрик.
В данном случае стоит опираться на соотношение уровня достатка и суммы кредита, а так же размера ежемесячного платежа.

<div class="alert alert-block alert-danger">
<b>Комментарий ревьюера❌:</b> В конце проекта принято кратко описывать все проделанные шаги и полученные результаты. Зачем это нужно - когда проект захочет посмотреть будущий работодатель, у него может не быть времени на подробный разбор кода. Вероятнее всего он бегло просмотрит код, но захочет изучить результат, который будет в общем выводе. Поэтому все же советую написать общий вывод)
</div>

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера_v2✅:</b> Отличный вывод, теперь работа логчески завершена и мы можем понять суть исследования🔥 </div>

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера✅:</b>Мне очень понравилась твоя работа. Все рассуждения верные, очень логичные и осмысленные. Ничего лишнего, всё по полкам. Ты подробно разбираешься в деталях, для будущего аналитика это очень важная черта😊

Проект еще нужно немного доработать, а именно:

- добавить описание проекта в самом начале
    
- исправить блок ответов на вопросы
    
- исправить общий вывод
    
Остальные замечания оставляю на твое усмотрение😉

Жду твою работу на повторное ревью.
    
[Тут есть полезные приемы Markdown](https://paulradzkov.com/2014/markdown_cheatsheet/)

</div>

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера_v2✅:</b> Отличная работа, проект принимаю. Могу с уверенностью сказать - ты хорошо владеешь пройденным материалом. Мне понравилась твоя реакция на обратную связь, комментарии даже к желтым замечаниям. Желаю удачи в дальнейшей учебе🔥 </div>